In [1]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

2024-01-16 23:33:05.487310: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 23:33:05.509431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 23:33:05.509452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 23:33:05.509960: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 23:33:05.513379: I tensorflow/core/platform/cpu_feature_guar

In [2]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.head()
x.set_index('DATE', inplace=True)
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [3]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
x_scale = x_scale.drop(columns=['Humidity', 'Precipitation', 'Wind Speed'])
x_scale.head()

,Temperature
DATE,
1981-07-01,0.926990
1981-07-02,-1.305674
1981-07-03,-1.367448
1981-07-04,-0.452614
1981-07-05,0.341614


In [4]:
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
time_steps = 1

X, y = create_dataset(x_scale, time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)
X_train.shape
X_val.shape

(3106, 1, 1)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(512, return_sequences=True, input_shape=(time_steps, 1)))
model.add(tf.keras.layers.LSTM(256, return_sequences=True))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(64, return_sequences=True))
model.add(tf.keras.layers.LSTM(32, return_sequences=True))
model.add(tf.keras.layers.LSTM(16, return_sequences=True))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, 'linear'))

cp = tf.keras.callbacks.ModelCheckpoint('model', save_best_only=True)
#model.compile(loss='mse', optimizer='adam', metrics=['rmse'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 1, 512)            1052672   
                                                                 
 lstm_8 (LSTM)               (None, 1, 256)            787456    
                                                                 
 lstm_9 (LSTM)               (None, 1, 128)            197120    
                                                                 
 lstm_10 (LSTM)              (None, 1, 64)             49408     
                                                                 
 lstm_11 (LSTM)              (None, 1, 32)             12416     
                                                                 
 lstm_12 (LSTM)              (None, 1, 16)             3136      
                                                                 
 dense_2 (Dense)             (None, 1, 8)             

In [15]:
model.compile(loss='Huber', optimizer='adam', metrics=['mae'])

In [16]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=cp)

Epoch 1/10
389/389 [==============================] - 9s 14ms/step - loss: 0.3816 - mae: 0.7444
Epoch 2/10
389/389 [==============================] - 6s 15ms/step - loss: 0.3658 - mae: 0.7255
Epoch 3/10
389/389 [==============================] - 6s 16ms/step - loss: 0.3648 - mae: 0.7234
Epoch 4/10
389/389 [==============================] - 6s 16ms/step - loss: 0.3646 - mae: 0.7239
Epoch 5/10
389/389 [==============================] - 6s 16ms/step - loss: 0.3643 - mae: 0.7229
Epoch 6/10
389/389 [==============================] - 6s 16ms/step - loss: 0.3644 - mae: 0.7232
Epoch 7/10
389/389 [==============================] - 4s 11ms/step - loss: 0.3645 - mae: 0.7233
Epoch 8/10
389/389 [==============================] - 5s 12ms/step - loss: 0.3639 - mae: 0.7226
Epoch 9/10
389/389 [==============================] - 6s 15ms/step - loss: 0.3643 - mae: 0.7232
Epoch 10/10
389/389 [==============================] - 6s 16ms/step - loss: 0.3644 - mae: 0.7231


In [17]:
matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [18]:
prediction = model.predict(X_val)
prediction_reshape = prediction[:, 0]
x_val_reshz = X_val[:, 0, 0]

98/98 [==============================] - 1s 3ms/step


In [20]:
time_val = x.index[-len(X_val):]

In [21]:
plt.plot(x_scale['Temperature'], color='green')
plt.plot(time_val, x_val_reshz, label='Actual', color='blue')
plt.plot(time_val, prediction_reshape, label='Prediction', color='orange')
plt.show()